# MANAGED / ABSTRACTED RAG USING THE GOOGLE GEMINI FILESEARCH API

- https://blog.google/technology/developers/file-search-gemini-api/


# Verify kernel selection

In [ ]:
# Run in a notebook cell and check that it gives the same output as `which python` in terminal
import sys
print(sys.executable)

/Users/zubinpratap/Documents/my_code/research-public/.venv/bin/python


# Install the libraries


In [ ]:
!uv pip install -U google-genai dotenv


In [43]:
from dotenv import load_dotenv
import os

from google import genai
from google.genai import types
import time

load_dotenv(override=True)
if not os.getenv("GEMINI_API_KEY"):
    print("GEMINI_API_KEY is not set")

client = genai.Client()

In [ ]:


# Create the File Search store with an optional display name
FILESTORE_NAME = 'research-public-filestore'
file_search_store = client.file_search_stores.create(
    config={'display_name': FILESTORE_NAME})

# Upload and import a file into the File Search store, supply a file name which will be visible in citations
upload_op = client.file_search_stores.upload_to_file_search_store(
    file='./zplinkedin_profile.pdf',  # TODOs swap for another one
    file_search_store_name=file_search_store.name,
    config={
        'display_name': 'ZP LinkedIn Profile',
        'custom_metadata': [
            {'key': 'source', 'string_value': 'https://www.linkedin.com/in/zubinpratap'},
            {'key': 'description', 'string_value': "Zubin Pratap's LinkedIn profile"}
        ]
    }
)

# Wait until import is complete
while not upload_op.done:
    print("sleeping....")
    time.sleep(5)
    upload_op = client.operations.get(upload_op)

print(" done? ", upload_op.done)

None


In [44]:

SYSTEM_PROMPT = """You are a helpful assistant. You tend to be brief, direct and simple"""

# Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""Can you tell me about Zubin""",
    config=types.GenerateContentConfig(
        system_instruction=SYSTEM_PROMPT,
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

print(response.text)

Zubin Pratap is a DevRel Engineering Leader, previously at Google, and currently at Chainlink Labs in Melbourne, Victoria, Australia. He is also a career transformation coach and a former lawyer.

**Career Overview:**
*   **Current Role:** DevRel Engineering Manager at Chainlink Labs (January 2025 - Present), preceded by a role as Technical Lead (April 2024 - February 2025).
*   **Previous Experience:** He spent 14 years as a corporate lawyer, handling global M&A deals, and later commercializing emerging technologies like SaaS, IoT, and drones. He also worked as a software engineer at Google, starting during the pandemic.
*   **Entrepreneurship:** Founded a startup in 2017 called Whooshka.me, which aimed to help users find street parking. He also founded NobleGenie (March 2014 - November 2014), a platform connecting users with qualified experts for specialized information.
*   **Career Pivot:** At 37, he taught himself to code, transitioning from law to tech.
*   **Coaching:** Since 20